In [1]:
#!/usr/bin/env python
from __future__ import print_function

from datetime import datetime
from collections import OrderedDict
import random
import os
import pandas as pd
from re import A, S
from numpy import maximum


import axioma
from axioma.assetset import AssetSet, ActionEntry
from axioma.account import Account
from axioma.accountgroup import AccountGroup
from axioma.workspace import DerbyProvider, Workspace
from axioma.workspace_element import ElementType
from axioma.costmodel import CostModel, CostStructure
from axioma.group import Group, Benchmark, Unit
from axioma.contentbuilder_group import ContentBuilderBenchmark, ContentBuilderGroup
from axioma.riskmodel import RiskModel
from axioma.strategy import Strategy, Objective, Target, Scope, MarketImpactType, PenaltyType
from axioma.rebalancing import Rebalancing, RebalancingStatus
from axioma.multiportfolio_rebalancing import MultiPortfolioRebalancing
from axioma.workspace_element import ElementType
from axioma.metagroup import Metagroup, DynamicMetagroup
from axioma.analytics import Analytics
import axioma.workspace_io as handler




In [2]:
axioma.ENDPOINT="http://localhost:8085/axioma-websrv"

In [ ]:




try:
    if 'ws' in locals() and ws is not None:
        try:
            ws.destroy()
        except Exception as e:
            # AxiomaError when server object is already gone; ignore that case
            if "ResourceNotFoundException" not in str(e):
                raise
finally:
    if 'ws' in locals():
        del ws



dates = ["2024-10-01","2024-10-02","2024-10-03"]

model = "US51AxiomaSH"

# iterate through dates in dates list

############################## CREATE WORKSPACE
i=1
date = datetime.strptime(dates[i], '%Y-%m-%d').date()
next_period_date = datetime.strptime(dates[i+1], '%Y-%m-%d').date()
axioma_data_dir = r'C:/Users/jwbpa/Box/Altisma_Data/AxiomaDownloader-3.3.0-with-jre64/output/extractedFiles/database/${yyyy}/${mm}/'
db_provider = DerbyProvider(axioma_data_dir,
                            risk_models=model,
                            include_composites=True,
                            next_period_date=next_period_date,
                            returns_type="Gross Return")

ws = Workspace(f"BetaCompletion_Day1_US51SH_{dates[i]}", date, data_provider=db_provider)

############################## LOAD DATA

ac_df = pd.read_csv(f"C:/Users/jwbpa/Box/Altisma_Data/Axioma/Example_Optimization/AlphaCapture_Optimized_{dates[i]}.csv", index_col="Name")
ac_account = Account(workspace = ws, 
        identity = "AC_Input", 
        holdings=ac_df.to_dict()["Shares"], 
        asset_map="Ticker Map")

ae_df = pd.read_csv(f"C:/Users/jwbpa/Box/Altisma_Data/Axioma/Example_Optimization/big_portfolio_{dates[i-1]}.csv", index_col="Name")
ae_account = Account(workspace = ws, 
        identity = "AE_Input", 
        holdings=ae_df.to_dict()["Shares"], 
        asset_map="Ticker Map")



net_account=AccountGroup(workspace=ws, identity = "AE_AC_AGGREGATE")
net_account.add_account(ac_account)
net_account.add_account(ae_account)


net_long_value = net_account.get_long_value(price_group="Price", exclude_futures=False)
net_short_value = net_account.get_short_value(price_group="Price", exclude_futures=False)
net_cash_value = net_account.get_total_cash_value(price_group="Price")
net_gross_value=net_long_value+net_short_value+net_cash_value

ac_long_value = ac_account.get_long_value(price_group="Price", exclude_futures=False)
ac_short_value = ac_account.get_short_value(price_group="Price", exclude_futures=False)
ac_cash_value = ac_account.get_total_cash_value(price_group="Price")
ac_gross_value=ac_long_value+ac_short_value+ac_cash_value

ae_long_value = ae_account.get_long_value(price_group="Price", exclude_futures=False)
ae_short_value = ae_account.get_short_value(price_group="Price", exclude_futures=False)
ae_cash_value = ae_account.get_total_cash_value(price_group="Price")
ae_gross_value=ae_long_value+ae_short_value+ae_cash_value


# Benchmark
handler.load_assets_from_data_provider(workspace=ws, asset_names=["SPY"], asset_map="Ticker Map")
# SPY benchmark is referenced via its composition; explicit asset load is unnecessary
spy_benchmark = ContentBuilderBenchmark(workspace=ws, identity = "SPY_Benchmark", expression = "'Composition of 37P4NKR33'*1")


## Content Builder Attributes
# Content Builder for 60-Day MDV
Inv_60_Day_MDV = ContentBuilderGroup(workspace=ws, identity = "Inv_60_Day_MDV", expression = "(1/'60-Day MDV')")

# Content Builder for Account Currency
Account_Currency = ContentBuilderGroup(workspace=ws, identity = "Account_Currency", expression = "portfolioAsCurrency('AC_Input')*1")

In [ ]:
try:
    if 'strategy' in locals() and strategy is not None:
        try:
            strategy.destroy()
        except Exception as e:
            # AxiomaError when server object is already gone; ignore that case
            if "ResourceNotFoundException" not in str(e):
                raise
finally:
    if 'strategy' in locals():
        del strategy


############################## DEFINE STRATEGY
strategy = Strategy(workspace = ws, 
                    identity = "Strategy", 
                    allow_shorting=True, 
                    allow_crossover=False, 
                    allow_grandfathering=True,
                    enable_constraint_hierarchy=True)
strategy.set_local_universe(local_universe=list(ac_account.get_holdings().keys()) + ["CSH_USD__"])

############################## DEFINE OBJECTIVE TERMS
ar_term = axioma.strategy.create_risk_term(strategy = strategy, 
                                            identity = "activeRisk", 
                                            benchmark=Account_Currency,
                                            factor_weight=0, 
                                            risk_model=model,
                                            specific_weight=1.0,
                                            asset_set="MASTER",
                                            qualification=ac_account
                                            )
tc_term = axioma.strategy.create_market_impact_term(strategy = strategy, 
                                                    identity = "marketImpact",
                                                    buy_impact_group=Inv_60_Day_MDV,
                                                    sell_impact_group=Inv_60_Day_MDV,
                                                    market_impact_type=MarketImpactType.Quadratic
                                                    )
############################## DEFINE OBJECTIVE FUNCTION
terms = OrderedDict()
terms[ar_term] = 1.0
terms[tc_term] = 1.0
obj_fx = Objective(strategy = strategy, 
                    identity = "Objective", 
                    terms=terms, 
                    target=Target.Minimize, 
                    active=True)
############################## DEFINE CONSTRAINTS

# constraint 1
constraint_do_not_trade_ae=axioma.strategy.create_limit_trade_constraint(strategy = strategy,
                                                              identity = "constraintDoNotTradeAE",
                                                              minimum=0,
                                                              maximum=0,
                                                              scope=Scope.Asset,
                                                              unit=Unit.Percent
)
# add multiselection qualification based on ae_df
constraint_do_not_trade_ae.add_selection(element_type=ElementType.AssetSet, element="MASTER", qualification=ae_account)


# constraint 2
constraint_market=axioma.strategy.create_limit_holding_constraint(strategy = strategy,
                                                                                 identity = "constraintMarket",
                                                                                 minimum=-0.1,
                                                                                 maximum=0.1,
                                                                                 benchmark=spy_benchmark,
                                                                                 scope=Scope.Selection,
                                                                                 unit=Unit.Percent
                                                                                 )

# add Market Intercept factor 
market_mgp = ws.get_metagroup(f"{model}.Market")
for group in market_mgp.get_groups():
    constraint_market.add_selection(element_type=ElementType.Group, element=group)



# constraint 3
constraint_country=axioma.strategy.create_limit_holding_constraint(strategy = strategy,
                                                                                 identity = "constraintCountry",
                                                                                 minimum=-0.1,
                                                                                 maximum=0.1,
                                                                                 benchmark=spy_benchmark,
                                                                                 scope=Scope.Selection,
                                                                                 unit=Unit.Percent
                                                                                 )

country_mgp = ws.get_metagroup(f"{model}.Country")
for group in country_mgp.get_groups():
    constraint_country.add_selection(element_type=ElementType.Group, element=group)



# constraint 4
constraint_local=axioma.strategy.create_limit_holding_constraint(strategy = strategy,
                                                                                 identity = "constraintLocal",
                                                                                 minimum=-0.1,
                                                                                 maximum=0.1,
                                                                                 benchmark=spy_benchmark,
                                                                                 scope=Scope.Selection,
                                                                                 unit=Unit.Percent
                                                                                 )

local_mgp = ws.get_metagroup(f"{model}.Local")
for group in local_mgp.get_groups():
    constraint_local.add_selection(element_type=ElementType.Group, element=group)



# constraint 5
constraint_currency=axioma.strategy.create_limit_holding_constraint(strategy = strategy,
                                                                                 identity = "constraintCurrency",
                                                                                 minimum=-0.1,
                                                                                 maximum=0.1,
                                                                                 benchmark=spy_benchmark,
                                                                                 scope=Scope.Selection,
                                                                                 unit=Unit.Percent
                                                                                 )

currency_mgp = ws.get_metagroup(f"{model}.Currency")
for group in currency_mgp.get_groups():
    constraint_currency.add_selection(element_type=ElementType.Group, element=group)





# constraint 6
constraint_style=axioma.strategy.create_limit_holding_constraint(strategy = strategy,
                                                                                 identity = "constraintStyle",
                                                                                 minimum=-0.1,
                                                                                 maximum=0.1,
                                                                                 benchmark=spy_benchmark,
                                                                                 scope=Scope.Selection,
                                                                                 unit=Unit.Percent
                                                                                 )


style_mgp = ws.get_metagroup(f"{model}.Style")
for group in style_mgp.get_groups():
    constraint_style.add_selection(element_type=ElementType.Group, element=group)


# constraint 7
constraint_sector=axioma.strategy.create_limit_holding_constraint(strategy = strategy,
                                                                                 identity = "constraintSector",
                                                                                 minimum=-0.1,
                                                                                 maximum=0.1,
                                                                                 benchmark=spy_benchmark,
                                                                                 scope=Scope.Selection,
                                                                                 unit=Unit.Percent
                                                                                 )

# add sector factors to multiselection except real estate and consumer discretionary
sectors = ws.get_metagroup(f"{model}.Sectors") 
sectors_metagroup_names_ex_real_estate_and_consumer_discretionary = [
    i.identity
    for i in sectors.get_metagroups()
    if i.identity != f"{model}.Real Estate-S" and i.identity != f"{model}.Consumer Discretionary-S"
]

for sector in sectors_metagroup_names_ex_real_estate_and_consumer_discretionary:
    constraint_sector.add_selection(element_type=ElementType.Metagroup, element=sector)








# constraint 8
constraint_realestate=axioma.strategy.create_limit_holding_constraint(strategy = strategy,
                                                                                 identity = "constraintRealestate",
                                                                                 minimum=-0.1,
                                                                                 maximum=0.1,
                                                                                 benchmark=spy_benchmark,
                                                                                 scope=Scope.Selection,
                                                                                 unit=Unit.Percent
                                                                                 )

# add real estate  to multiselection
constraint_realestate.add_selection(element_type=ElementType.Metagroup, element=f"{model}.Real Estate-S")


# constraint 9
constraint_consumerdiscretionary=axioma.strategy.create_limit_holding_constraint(strategy = strategy,
                                                                                 identity = "constraintConsumerdiscretionary",
                                                                                 minimum=-0.1,
                                                                                 maximum=0.1,
                                                                                 benchmark=spy_benchmark,
                                                                                 scope=Scope.Selection,
                                                                                 unit=Unit.Percent
                                                                                 )

# add consumer discretionary to multiselection
constraint_consumerdiscretionary.add_selection(element_type=ElementType.Metagroup, element=f"{model}.Consumer Discretionary-S")




# constraint 10
constraint_factor_aggregate=axioma.strategy.create_limit_holding_constraint(strategy = strategy,
                                                                                 identity = "constraintFactorAggregate",
                                                                                 minimum=-0.5,
                                                                                 maximum=0.5,
                                                                                 benchmark=spy_benchmark,
                                                                                 scope=Scope.Aggregate,
                                                                                 unit=Unit.Percent
                                                                                 )

# add all factors to multiselection
rm = ws.get_risk_model(model)
for factor in rm.get_factor_names():
    constraint_factor_aggregate.add_selection(element_type=ElementType.Group, element=f"{model}.{factor}")




# constraint 7
budget=axioma.strategy.create_budget_constraint(strategy = strategy,
                                                identity = "budget",
                                                qualification=ac_account,
                                                use_budget_value=True
                                                )




# constraint 8
empty_benchmark = Benchmark(workspace = ws, identity = "empty benchmark", values={}, unit = Unit.Currency)


sum_squared_weights=axioma.strategy.create_limit_model_deviation_constraint(strategy = strategy,
                                                                            identity = "sumSquaredWeights",
                                                                            maximum=25,
                                                                            penalty=400,
                                                                            benchmark=empty_benchmark,
                                                                            penalty_type=PenaltyType.ScaledQuadratic,
                                                                            scope=Scope.Selection,
                                                                            unit=Unit.Percent
                                                                            )

# add NON-CASH ASSETS to selection
sum_squared_weights.add_selection(element_type=ElementType.AssetSet, element="NON-CASH ASSETS")


############################## SET CONSTRAINT HIERARCHY CONSTRAINTS

# strategy.set_constraint_hierarchy({
#     "matchBenchmarkRealestateSectorExposure": 1,
#     "matchBenchmarkConsumerDiscretionarySectorExposure": 2,
#     "matchBenchmarkSectorExposure":3,
#     "sumSquaredWeights": 4
# })

try:
    if 'rebal' in locals() and rebal is not None:
        try:
            rebal.destroy()
        except Exception as e:
            # AxiomaError when server object is already gone; ignore that case
            if "ResourceNotFoundException" not in str(e):
                raise
finally:
    if 'rebal' in locals():
        del rebal






############################## CREATE REBALANCING
rebal = MultiPortfolioRebalancing(workspace = ws, 
                    identity="BetaCompletion_Rebalancing", 
                    strategy=strategy
                    )

rebal.add_account(ac_account)
rebal.add_account(ae_account)

rebal.set_reference_size(net_gross_value)
#rebal.set_budget_size(1000000)
#rebal.set_benchmark_size(1000000)


rebal.set_rebalancing_defaults(round_lot_size=1,
                               use_cash_for_roundlot=True,
                               risk_model=model)



In [151]:
# === ONE-CELL: discover real constraint names -> ordered relaxation (minimal set) -> optional budget diagnostic ===
import re
from collections import OrderedDict

# Expect these to exist already:
try:
    _strat = strategy
    _rebal = rebal
except NameError as e:
    raise RuntimeError("This cell expects 'strategy' and 'rebal' to be defined before running.") from e

# ---------- status helpers (robust with/without enum import) ----------
def _ok_status(st):
    s = str(st)
    return ("SolutionFound" in s) or ("RelaxedSolutionFound" in s)

def _status_name(st):
    s = str(st)
    return s.split(".")[-1] if "." in s else s

# ---------- Your policy ----------
# Keep these HARD (never relax). Use your *real* identities:
NEVER_RELAX = {}  # 'BudgetConstraint' alias is not used in your notebook; real name is 'budget'

# Your preferred order (names may be fuzzy-resolved to actual identities discovered below)
PREFERRED_ORDER = [
    "sumSquaredWeights",
    "matchBenchmarkRealestateSectorExposure",
    "matchBenchmarkConsumerDiscretionarySectorExposure",
    "matchBenchmarkSectorExposure",
    "matchBenchmarkStyleExposure",
    "matchBenchmarkMarketExposure",
    "matchBenchmarkAggregateExposure", 
    "doNotTradeAE",
    'budget' # last resort
]

# Optional diagnostic to quantify budget mismatch if still infeasible with hard budget
ALLOW_TEMP_BUDGET_DIAG = False  # <-- set to True ONLY to measure how far budget would need to move (does NOT keep it relaxed in final solve)

# ---------- Make hierarchy strict & clear any prior hierarchy ----------
try:
    _strat.set_options(enable_constraint_hierarchy=True)
except Exception:
    pass

# Avoid reusing softened solutions; avoid auto micro-slack
try:
    _rebal.set_rebalancing_options(use_memento=False, hierarchy_tolerance_value=0.0)
except TypeError:
    try: _rebal.set_rebalancing_options(use_memento=False)
    except Exception: pass
    try: _rebal.set_rebalancing_options(hierarchy_tolerance_value=0.0)
    except Exception: pass
except Exception:
    pass

try:
    _strat.set_constraint_hierarchy({})
except Exception:
    pass

# ---------- Discovery: pull actual constraint objects and identities ----------
def _identity_from_obj(c):
    # Try attributes
    for attr in ("identity","name","id","_identity","_name"):
        if hasattr(c, attr):
            v = getattr(c, attr)
            if callable(v):
                try: v = v()
                except Exception: v = None
            if isinstance(v, str) and v.strip():
                return v.strip()
    # Try common methods
    for m in ("get_identity","get_name","getId"):
        if hasattr(c, m):
            try:
                v = getattr(c, m)()
                if isinstance(v, str) and v.strip():
                    return v.strip()
            except Exception:
                pass
    # Try to_dict
    if hasattr(c, "to_dict"):
        try:
            d = c.to_dict()
            for k in ("identity","name","id"):
                if k in d and isinstance(d[k], str) and d[k].strip():
                    return d[k].strip()
        except Exception:
            pass
    return None

def discover_constraints(strategy):
    discovered = []
    # 1) get_constraints()
    if hasattr(strategy, "get_constraints"):
        try:
            for c in (strategy.get_constraints() or []):
                nm = _identity_from_obj(c)
                discovered.append((nm, c.__class__.__name__))
        except Exception:
            pass
    # 2) fallback maps/attrs (very defensive; may duplicate)
    for meth in ("get_constraint_map","get_constraint_names","list_constraints"):
        if hasattr(strategy, meth):
            try:
                obj = getattr(strategy, meth)()
                if isinstance(obj, dict):
                    for k, v in obj.items():
                        name = str(k) if k is not None else None
                        if name: discovered.append((name, getattr(v, "__class__", type(v)).__name__))
                elif isinstance(obj, (list, tuple, set)):
                    for x in obj:
                        if isinstance(x, str):
                            discovered.append((x, ""))
                        else:
                            nm = _identity_from_obj(x)
                            discovered.append((nm, getattr(x, "__class__", type(x)).__name__))
            except Exception:
                pass

    # Clean
    clean = []
    for nm, tp in discovered:
        if nm and isinstance(nm, str):
            clean.append((nm, tp))
    # Deduplicate by name while preferring the first seen type
    seen, rows = set(), []
    for nm, tp in clean:
        if nm not in seen:
            rows.append((nm, tp))
            seen.add(nm)
    return rows

rows = discover_constraints(_strat)

print("\n=== DISCOVERED CONSTRAINT IDENTITIES (from strategy) ===")
if not rows:
    print("(None discovered; API may hide them on this object. Continue anyway.)")
else:
    w = max(len(r[0]) for r in rows)
    for nm, tp in rows:
        print(f"  {nm.ljust(w)}  [{tp}]")

DISCOVERED_NAMES = [nm for nm,_ in rows]

# ---------- Resolve your preferred names to real identities (lenient match) ----------
def _norm(s): return re.sub(r"[^a-z0-9]+", "", s.lower()) if isinstance(s, str) else ""
disc_norm = { _norm(nm): nm for nm in DISCOVERED_NAMES }

resolved = []
unresolved = []
for wanted in PREFERRED_ORDER:
    if wanted in DISCOVERED_NAMES:
        resolved.append(wanted)
        continue
    wn = _norm(wanted)
    match = disc_norm.get(wn)
    if not match:
        # lenient: substring either way
        for dn_raw in DISCOVERED_NAMES:
            dn = _norm(dn_raw)
            if wn and (wn in dn or dn in wn):
                match = dn_raw
                break
    if match:
        resolved.append(match)
    else:
        unresolved.append(wanted)

# Remove hard set and duplicates from resolved
RESOLVED_ORDER = []
seen = set()
for nm in resolved:
    if nm in NEVER_RELAX:  # never relax
        continue
    if nm not in seen:
        RESOLVED_ORDER.append(nm); seen.add(nm)

print("\nResolved preferred names to real identities (order):", RESOLVED_ORDER)
if unresolved:
    print("Could not resolve (check your naming):", unresolved)

# ---------- Helpers for hierarchy/solve/report ----------
def _apply_hierarchy(names_in_order):
    mapping = OrderedDict((n, i+1) for i, n in enumerate(names_in_order))
    _strat.set_constraint_hierarchy(mapping)

def _solve():
    return _rebal.solve()

def _ordered_relax_minimal(order_names):
    # Start with nothing relaxable
    try: _apply_hierarchy([])
    except Exception: pass

    relaxed, sol = [], None

    # Add one-by-one
    for nm in order_names:
        relaxed.append(nm)
        _apply_hierarchy(relaxed)
        try:
            s_try = _solve()
            if _ok_status(s_try.get_status()):
                sol = s_try
                break
        except Exception:
            sol = None
            continue

    # If still none, try all at once (in this order)
    if sol is None and order_names:
        _apply_hierarchy(order_names)
        try:
            s_try = _solve()
            if _ok_status(s_try.get_status()):
                sol, relaxed = s_try, list(order_names)
            else:
                return None, []
        except Exception:
            return None, []

    # Reverse-harden to minimal
    for nm in list(relaxed)[::-1]:
        trial = [x for x in relaxed if x != nm]
        _apply_hierarchy(trial)
        try:
            s2 = _solve()
            if _ok_status(s2.get_status()):
                sol, relaxed = s2, trial
            else:
                _apply_hierarchy(relaxed)  # restore
        except Exception:
            _apply_hierarchy(relaxed)      # restore

    return sol, relaxed

def _report_hard_violations(solution, focus=None, top_per_constraint=5, base_order=None):
    try:
        rows = solution.get_constraint_values()
    except Exception as e:
        print("Constraint values unavailable:", e)
        return
    if focus:
        fset = {n.lower() for n in focus}
        rows = [r for r in rows if str(r.get("constraint_name","")).lower() in fset]
    by = {}
    for r in rows:
        by.setdefault(r.get("constraint_name"), []).append(r)

    def _order_key(k):
        if base_order and k in base_order: return base_order.index(k)
        return 10**9

    print("\n=== PER-CONSTRAINT hard_violation (vs ORIGINAL hard bounds) ===")
    if not by:
        print("(No rows returned for the requested constraints.)")
        return

    for cname in sorted(by.keys(), key=_order_key):
        items = by[cname]
        worst = max(items, key=lambda d: abs(d.get("hard_violation") or 0.0))
        unit  = worst.get("constraint_unit")
        max_hv = abs(worst.get("hard_violation") or 0.0)
        print(f"\n[{cname}]  max hard_violation={max_hv:.6g} {unit or ''}")
        for r in sorted(items, key=lambda d: abs(d.get("hard_violation") or 0.0), reverse=True)[:top_per_constraint]:
            print(
                f"  • {r.get('selection_name')}: "
                f"final={r.get('final_value')}  "
                f"applied=({r.get('applied_min')},{r.get('applied_max')})  "
                f"violation={r.get('violation')}  "
                f"hard_violation={r.get('hard_violation')} {unit or ''}"
            )
# === PATCH: robust "hard-violation" reporter for Axioma ConstraintValue objects OR dicts ===
import re

def _normalize_key(k: str) -> str:
    return re.sub(r"[^a-z0-9]+", "", (k or "").lower())

def _safe_call(v):
    try:
        return v() if callable(v) else v
    except Exception:
        return v

def _lookup_attr(row, candidates):
    """
    Try several attribute/method spellings on an object or dict, then do a
    case/underscore-insensitive scan of available attributes as a last resort.
    """
    # 1) direct hit on dict
    if isinstance(row, dict):
        for c in candidates:
            if c in row:
                return row[c]
        # also try normalized match
        norm_map = {_normalize_key(k): k for k in row.keys()}
        for c in candidates:
            nk = _normalize_key(c)
            if nk in norm_map:
                return row[norm_map[nk]]
        return None

    # 2) direct attribute/method on object
    for c in candidates:
        if hasattr(row, c):
            return _safe_call(getattr(row, c))

    # 3) "getXxx" method variants
    for c in candidates:
        cc = c[0].upper() + c[1:] if c else c
        for alt in (f"get_{c}", f"get{cc}"):
            if hasattr(row, alt):
                return _safe_call(getattr(row, alt))

    # 4) normalized attribute name search
    try:
        names = dir(row)
        norm_map = {_normalize_key(n): n for n in names}
        for c in candidates:
            nk = _normalize_key(c)
            if nk in norm_map:
                return _safe_call(getattr(row, norm_map[nk]))
    except Exception:
        pass

    # 5) to_dict / toDict fallback
    for m in ("to_dict", "toDict"):
        if hasattr(row, m):
            try:
                d = getattr(row, m)()
                if isinstance(d, dict):
                    return _lookup_attr(d, candidates)
            except Exception:
                pass

    return None

def _row_to_dict(row):
    """
    Normalize a single row (dict OR object) to a dict with consistent keys.
    """
    fields = {
        "constraint_name": ["constraint_name","constraintName","name","constraint","constraint_id","constraintId"],
        "selection_name":  ["selection_name","selectionName","selection","bucket","asset","group","groupName"],
        "final_value":     ["final_value","finalValue","value","final","actualValue","measureValue"],
        "applied_min":     ["applied_min","appliedMin","min","appliedLower","lower","lowerBound","appliedLowerBound"],
        "applied_max":     ["applied_max","appliedMax","max","appliedUpper","upper","upperBound","appliedUpperBound"],
        "violation":       ["violation","applied_violation","appliedViolation","slack"],
        "hard_violation":  ["hard_violation","hardViolation","hardSlack","violation_vs_original","violationVsOriginal"],
        "constraint_unit": ["constraint_unit","constraintUnit","unit","units"],
        "side":            ["side"]
    }
    out = {}
    for k, cands in fields.items():
        out[k] = _lookup_attr(row, cands)
    return out

def _report_hard_violations(solution, focus=None, top_per_constraint=5, base_order=None):
    """
    Print compact per-constraint report of 'hard_violation' (vs ORIGINAL hard bounds),
    robust to Axioma returning dicts OR 'ConstraintValue' objects.
    """
    # 1) fetch rows
    rows = []
    try:
        rows = solution.get_constraint_values()
    except Exception as e:
        print("get_constraint_values() failed:", e)
        rows = []

    # 2) normalize rows to dicts
    norm = [_row_to_dict(r) for r in (rows or [])]
    norm = [r for r in norm if r.get("constraint_name")]

    # 3) optional focus filter
    if focus:
        fset = {str(n).lower() for n in focus}
        norm = [r for r in norm if str(r.get("constraint_name","")).lower() in fset]

    # 4) group by constraint
    by = {}
    for r in norm:
        by.setdefault(r["constraint_name"], []).append(r)

    # 5) nothing to show?
    print("\n=== PER-CONSTRAINT hard_violation (vs ORIGINAL hard bounds) ===")
    if not by:
        print("(No rows matched; verify constraint names in 'focus'.)")
        # show quick debug if we can
        sample = rows[0] if rows else None
        if sample is not None:
            print("Example row type:", type(sample))
            try:
                some = dir(sample)
                print("Example available attributes (head):",
                      [a for a in some if not a.startswith("_")][:25])
            except Exception:
                pass
        return by

    # 6) printing order helper
    if base_order is None:
        base_order = []
    def _order_key(name):
        try:
            return list(base_order).index(name)
        except ValueError:
            return 10**9

    # 7) emit report
    def _absfloat(x):
        try:
            return abs(float(x))
        except Exception:
            return 0.0

    for cname in sorted(by.keys(), key=_order_key):
        items = by[cname]
        worst = max(items, key=lambda d: _absfloat(d.get("hard_violation")))
        unit  = worst.get("constraint_unit") or ""
        print(f"\n[{cname}]  max hard_violation={_absfloat(worst.get('hard_violation')):.6g} {unit}")
        # Top offenders within the constraint
        top = sorted(items, key=lambda d: _absfloat(d.get("hard_violation")), reverse=True)[:top_per_constraint]
        for r in top:
            print("  • {sel}: final={fv}  applied=({mn},{mx})  violation={v}  hard_violation={hv} {u}".format(
                sel=r.get("selection_name"),
                fv=r.get("final_value"),
                mn=r.get("applied_min"),
                mx=r.get("applied_max"),
                v=r.get("violation"),
                hv=r.get("hard_violation"),
                u=unit
            ))
    return by





# ---------- RUN: your resolved order ----------
sol, relaxed = _ordered_relax_minimal(RESOLVED_ORDER)

# ---------- IF STILL infeasible: try adding any other discovered constraints (except hard set) ----------
if sol is None:
    extras = [nm for nm in DISCOVERED_NAMES if nm not in NEVER_RELAX and nm not in RESOLVED_ORDER]
    if extras:
        print("\nExpanding relax order with discovered constraints (excluding hard set):", extras)
        sol, relaxed = _ordered_relax_minimal(RESOLVED_ORDER + extras)

# ---------- Final report or diagnostic ----------
if sol is None:
    print("\n=== RESULT: Still infeasible even after relaxing resolved & discovered constraints (except hard set). ===")
    print("Likely a hard conflict (often budget vs doNotTradeAE or another hard limit like an exact cardinality).")
    # Optional diagnostic: temporarily allow budget to relax JUST to measure mismatch
    if ALLOW_TEMP_BUDGET_DIAG and ("budget" in DISCOVERED_NAMES):
        print("\n--- DIAGNOSTIC (temporary): allowing 'budget' to relax to measure mismatch ---")
        diag_order = ["budget"] + [nm for nm in (RESOLVED_ORDER + extras if 'extras' in locals() else RESOLVED_ORDER) if nm != "budget"]
        _apply_hierarchy(diag_order)
        try:
            s_diag = _solve()
            if _ok_status(s_diag.get_status()):
                print("Diagnostic solve succeeded only when 'budget' is relaxable. Here's how far it needed to move:")
                _report_hard_violations(s_diag, focus={"budget"})
                print("\n(End of diagnostic. Re-run with ALLOW_TEMP_BUDGET_DIAG=False for normal behavior.)")
            else:
                print("Diagnostic solve still infeasible even with 'budget' relaxable.")
        except Exception as e:
            print("Diagnostic solve failed:", e)
else:
    print("\n=== SUMMARY ===")
    print("Final status:", _status_name(sol.get_status()))
    print("Relaxed (minimal set):", relaxed)
    print("Hard (explicit):", sorted(NEVER_RELAX))
    # === Call it (same way you did before). If you don't have PREFERRED_ORDER defined, pass list(relaxed).
    _report_hard_violations(sol, focus=set(relaxed), base_order=PREFERRED_ORDER if "PREFERRED_ORDER" in globals() else list(relaxed))


    # Optional: trades (safe only when solved)
    try:
        try:
            trades = sol.get_trade_list(include_initial_holdings=False,
                                        include_multi_rebal_settings=False,
                                        sales_are_negative=True,
                                        split_crossover_trades=True)
        except TypeError:
            trades = sol.get_trade_list(include_initial_holdings=False,
                                        include_rebal_settings=False,
                                        sales_are_negative=True,
                                        split_crossover_trades=True)
        print("\nTrade list available. Keys:", list(trades)[:10])
    except Exception as e:
        print("Trades unavailable:", e)



=== DISCOVERED CONSTRAINT IDENTITIES (from strategy) ===
  budget                                             [Constraint]
  doNotTradeAE                                       [Constraint]
  matchBenchmarkAggregateExposure                    [Constraint]
  matchBenchmarkConsumerDiscretionarySectorExposure  [Constraint]
  matchBenchmarkMarketExposure                       [Constraint]
  matchBenchmarkRealestateSectorExposure             [Constraint]
  matchBenchmarkSectorExposure                       [Constraint]
  matchBenchmarkStyleExposure                        [Constraint]
  sumSquaredWeights                                  [Constraint]

Resolved preferred names to real identities (order): ['sumSquaredWeights', 'matchBenchmarkRealestateSectorExposure', 'matchBenchmarkConsumerDiscretionarySectorExposure', 'matchBenchmarkSectorExposure', 'matchBenchmarkStyleExposure', 'matchBenchmarkMarketExposure', 'matchBenchmarkAggregateExposure', 'doNotTradeAE', 'budget']

=== SUMMARY ===
Fina

In [152]:
getters = ("get_constraint_names","list_constraints","get_constraints","get_constraint_map")
for g in getters:
    if hasattr(strategy, g):
        try:
            print(g, getattr(strategy, g)())
        except Exception:
            pass


get_constraints [<axioma.strategy.Constraint object at 0x0000021DFB2A4280>, <axioma.strategy.Constraint object at 0x0000021DFB2A54E0>, <axioma.strategy.Constraint object at 0x0000021DFB2A7CB0>, <axioma.strategy.Constraint object at 0x0000021DFB2A6820>, <axioma.strategy.Constraint object at 0x0000021DFCBFB1C0>, <axioma.strategy.Constraint object at 0x0000021DFCBF92B0>, <axioma.strategy.Constraint object at 0x0000021DFCBF9550>, <axioma.strategy.Constraint object at 0x0000021DFCBFB380>, <axioma.strategy.Constraint object at 0x0000021DFCBFB2A0>]


In [153]:
# After the cell above runs
# This is just the mapping we sent; Axioma doesn't always offer a getter:
print("Hierarchy applied (relaxable in priority order):", relaxed)
print("Hard constraints include:", sorted(NEVER_RELAX))


Hierarchy applied (relaxable in priority order): ['sumSquaredWeights', 'matchBenchmarkRealestateSectorExposure', 'matchBenchmarkConsumerDiscretionarySectorExposure', 'matchBenchmarkSectorExposure', 'matchBenchmarkStyleExposure', 'matchBenchmarkMarketExposure', 'matchBenchmarkAggregateExposure', 'doNotTradeAE']
Hard constraints include: []


In [ ]:
############################## SOLVE REBALANCING
sol = rebal.solve()
print(sol.get_status())
if sol.get_status()==RebalancingStatus.SolutionFound or sol.get_status()==RebalancingStatus.RelaxedSolutionFound:
    fh = sol.get_final_holdings(asset_map = "Ticker Map")
#sol.get_final_holdings(asset_map=None)
ae_fh=sol.get_final_account_holdings_for_account(ae_account, asset_map="Ticker Map")
ac_fh=sol.get_final_account_holdings_for_account(ac_account, asset_map="Ticker Map")

In [ ]:


############################## CALCULATE ANALYTICS - THESE ARE EXAMPLES OF SOME AVAILABLE ANALYTICS
# create Analytics object. This will be used to calculate analytics
analyzer = Analytics(workspace = ws, price_group="Price", asset_map="Ticker Map")

# compute active holdings of final holdings
ah = analyzer.compute_active_holdings(holdings=fh, benchmark=spy_benchmark, reference_value=rebal.get_reference_size())

# calculate active exposures -> you divide df by reference size to convert from currency values to decimal
ah_exposures = pd.DataFrame.from_dict({"Exposure":analyzer.compute_factor_exposures(risk_model=model,holdings=ah)})/rebal.get_reference_size()

# compute tracking error.
tracking_error = analyzer.compute_factors_risk(risk_model=model,holdings=ah)/rebal.get_reference_size()*100


########################## Find Idio Return and Risk of final solution next day.
# Factor return
factor_return_dollar = analyzer.compute_factor_return_contributions(risk_model=model, 
                                                          metagroup = f"{model}.Period Returns",
                                                          holdings=ah)

# Total return
total_return_dollar =analyzer.compute_expected_return(alphas="Period Return", holdings=ah)

# Idio return
idio_return_dollar = total_return_dollar - factor_return_dollar

# Convert dollar to %
idio_return_percent = idio_return_dollar / rebal.get_reference_size()*100

# Factor risk
factor_risk_dollar = analyzer.compute_factors_risk(risk_model=model, holdings=ah)

# Total risk
total_risk_dollar = analyzer.compute_total_risk(risk_model=model, holdings=ah)

# Idio risk
idio_risk_dollar = analyzer.compute_specific_risk(risk_model=model, holdings=ah)

# Convert dollar to %
idio_risk_percent = idio_risk_dollar / rebal.get_reference_size()*100




# write workspace to .wsp file and release license token
ws.write(os.getcwd(),file_name=f"workspace_on_{dates[i]}.wsp",save_reference=False)
ws.destroy()
